In [3]:
# Adding autoreload will let you edit your package and see the changes without restarting the notebook.

%load_ext autoreload

%autoreload 2

In [25]:
import pandas as pd
from datetime import datetime

In [98]:
deliv_df = pd.read_json(path_or_buf="../data/raw/deliveries.jsonl", lines=True)
sess_df = pd.read_json(path_or_buf="../data/raw/sessions.jsonl", lines=True)
usr_df = pd.read_json(path_or_buf="../data/raw/users.jsonl", lines=True)
prod_df = pd.read_json(path_or_buf="../data/raw/products.jsonl", lines=True)

In [94]:
deliv_df

,purchase_id,purchase_timestamp,delivery_timestamp,delivery_company
0,20001,2021-10-22 00:55:31,2021-10-26T00:56:59,516.0
1,20002,2021-10-16 01:50:24,2021-10-20T01:51:42,620.0
2,20003,2021-10-02 12:42:52,2021-10-05T12:46:54,620.0
3,20004,2021-08-25 12:05:56,2021-08-28T12:07:40,516.0
4,20005,2021-08-01 22:35:46,2021-08-04T22:38:12,620.0
...,...,...,...,...
5177,25178,2021-08-10 02:04:58,2021-08-13T02:08:10,360.0
5178,25179,2021-08-27 15:51:13,2021-09-01T15:52:31,620.0
5179,25180,2021-10-14 14:59:51,2021-10-19T15:04:26,620.0
5180,25181,2021-09-05 17:36:21,2021-09-09T17:38:14,360.0


In [6]:
usr_df

,user_id,name,city,street
0,102,Inga Jeka,Gdynia,plac Wspólna 05
1,103,Anastazja Jarczak,Kraków,plac Poniatowskiego 92/19
2,104,Błażej Gabrych,Radom,aleja Wojska Polskiego 31
3,105,Klara Samulak,Poznań,ulica Skargi 17
4,106,Nikodem Kuffel,Poznań,ulica Piwna 02/41
...,...,...,...,...
195,297,Aurelia Waloszek,Wrocław,ulica Szczęśliwa 15/15
196,298,Kajetan Choina,Kraków,ul. Diamentowa 96/68
197,299,Cezary Dolega,Wrocław,aleja Kalinowa 15/46
198,300,Grzegorz Sadlik,Gdynia,pl. Jana III Sobieskiego 26


In [81]:
prod_df

,product_id,product_name,category_path,price
0,1001,Telefon Siemens Gigaset DA310,Telefony i akcesoria;Telefony stacjonarne,58.97
1,1002,Kyocera FS-1135MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,2048.50
2,1003,Kyocera FS-3640MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,7639.00
3,1004,Fallout 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99
4,1005,Szalone Króliki Na żywo i w kolorze (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99
...,...,...,...,...
314,1315,Jabra Talk,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,54.99
315,1316,Plantronics Voyager Legend,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,-249.00
316,1317,Plantronics Savi W740,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,1303.96
317,1318,Plantronics Savi W710,Sprzęt RTV;Audio;Słuchawki,553.00


In [82]:
sess_df

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id
0,100001,2021-08-30 18:06:31,102.0,1001.0,VIEW_PRODUCT,0,NaN
1,100002,2021-08-28 04:15:09,102.0,1037.0,VIEW_PRODUCT,20,NaN
2,100003,2021-08-24 16:20:45,102.0,1035.0,VIEW_PRODUCT,15,NaN
3,100003,2021-08-24 16:24:51,102.0,1067.0,VIEW_PRODUCT,15,NaN
4,100004,2021-10-22 00:50:51,102.0,1283.0,VIEW_PRODUCT,5,NaN
...,...,...,...,...,...,...,...
37241,110416,2021-09-05 17:32:12,301.0,1277.0,VIEW_PRODUCT,10,NaN
37242,110416,2021-09-05 17:33:30,301.0,1276.0,VIEW_PRODUCT,10,NaN
37243,110416,2021-09-05 17:36:21,301.0,1276.0,BUY_PRODUCT,10,25181.0
37244,110417,2021-07-30 11:52:43,301.0,1318.0,VIEW_PRODUCT,5,NaN


In [41]:
sess_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37246 entries, 0 to 37245
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   session_id        37246 non-null  int64         
 1   timestamp         37246 non-null  datetime64[ns]
 2   user_id           35463 non-null  float64       
 3   product_id        35413 non-null  float64       
 4   event_type        37246 non-null  object        
 5   offered_discount  37246 non-null  int64         
 6   purchase_id       5182 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 2.0+ MB


In [99]:
sess_df = sess_df[sess_df.user_id.notnull()].drop(["session_id", "purchase_id"], axis=1).reindex()
sess_df

,timestamp,user_id,product_id,event_type,offered_discount
0,2021-08-30 18:06:31,102.0,1001.0,VIEW_PRODUCT,0
1,2021-08-28 04:15:09,102.0,1037.0,VIEW_PRODUCT,20
2,2021-08-24 16:20:45,102.0,1035.0,VIEW_PRODUCT,15
3,2021-08-24 16:24:51,102.0,1067.0,VIEW_PRODUCT,15
4,2021-10-22 00:50:51,102.0,1283.0,VIEW_PRODUCT,5
...,...,...,...,...,...
37241,2021-09-05 17:32:12,301.0,1277.0,VIEW_PRODUCT,10
37242,2021-09-05 17:33:30,301.0,1276.0,VIEW_PRODUCT,10
37243,2021-09-05 17:36:21,301.0,1276.0,BUY_PRODUCT,10
37244,2021-07-30 11:52:43,301.0,1318.0,VIEW_PRODUCT,5


In [115]:
usr_df["is_male"] = usr_df.name.apply(lambda name_str: int(name_str.split()[0][-1]!="a"))

,user_id,name,city,street,is_male
0,102,Inga Jeka,Gdynia,plac Wspólna 05,0
1,103,Anastazja Jarczak,Kraków,plac Poniatowskiego 92/19,0
2,104,Błażej Gabrych,Radom,aleja Wojska Polskiego 31,1
3,105,Klara Samulak,Poznań,ulica Skargi 17,0
4,106,Nikodem Kuffel,Poznań,ulica Piwna 02/41,1
...,...,...,...,...,...
195,297,Aurelia Waloszek,Wrocław,ulica Szczęśliwa 15/15,0
196,298,Kajetan Choina,Kraków,ul. Diamentowa 96/68,1
197,299,Cezary Dolega,Wrocław,aleja Kalinowa 15/46,1
198,300,Grzegorz Sadlik,Gdynia,pl. Jana III Sobieskiego 26,1


In [118]:
usr_df.nunique()

user_id    200
name       200
city         7
street     200
is_male      2
dtype: int64

In [120]:
usr_df.drop(["name", "street"],axis=1, inplace=True)

SyntaxError: invalid syntax (144205692.py, line 1)